<a href="https://colab.research.google.com/github/varekarprajwal/DRIVE/blob/main/1337x_com_to_GD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<code>function ClickConnect(){
console.log("Working");
document.querySelector("colab-connect-button").click()
}setInterval(ClickConnect,6000)</code>

In [ ]:
#@title Module instal and Variable create
!python -m pip install --upgrade pip setuptools wheel
!python -m pip install lbry-libtorrent
!apt install python3-libtorrent
import re
from bs4 import BeautifulSoup
import requests
import pandas as pd
from google.colab import data_table
import libtorrent as lt
import urllib.request
from PIL import Image
data_table.enable_dataframe_formatter()
ses = lt.session()
ses.listen_on(6881, 6891)
downloads = []
movies_name=[]
movies_size_seed=[]
movies_upload_date=[]
link=[]
x=[]

In [ ]:
#@title Google Autorization
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [76]:
#@title Trending movie list
trending_torrent = "movies" #@param ["movies","anime","xxx"]
trending_type = "Today" #@param ["Today", "Week"]
del movies_name[::]
del movies_size_seed[::]
del movies_upload_date[::]
del link[::]
del x[::]
if trending_type == "Today":
  types="d"
else :types="w"
url="https://1337x.to/trending/"+types+"/"+trending_torrent+"/"
r2=requests.get(url)
soup=BeautifulSoup(r2.content,'html.parser')
table1=soup.find('div',class_='table-list-wrap').find('tbody').find_all('tr')
table_name=soup.find('div',class_='table-list-wrap')
x=table_name.find_all('td',class_='coll-1 name')

for i in range(len(table1)):
  # print(x[i].find('a').get_text())
  movies_name.append(x[i].find('a').get_text())
  movies_size_seed.append(table1[i].find_all('td')[4].get_text().replace("B","B "))
  movies_upload_date.append(table1[i].find_all('td')[3].get_text())
  temp= soup.find_all('td',class_='coll-1 name')
  link.append(temp[i].find('a', href=True)['href'])
data_trending = {'Movie Name': movies_name,
        'Size And Seed': movies_size_seed,
        'Upload':movies_upload_date}
df = pd.DataFrame(data_trending)
data_table.DataTable(df, num_rows_per_page=len(x))

,Movie Name,Size And Seed,Upload
0,Guardians of the Galaxy Vol 3 IMAX 2160p 4k 20...,5.9 GB 868,2pm Jul. 9th
1,Buddys Mom (2015) Explicit 720p HDRip x264 AAC...,759.4 MB 341,1pm Jul. 9th
2,LOLA.2023.720p.AMZN.WEBRip.800MB.x264-GalaxyRG,795.8 MB 455,2:08am
3,LOLA.2023.1080p.AMZN.WEBRip.1400MB.DD5.1.x264-...,1.4 GB 329,2:31am
4,Gods.Country.Song.2023.720p.AMZN.WEBRip.800MB....,796.0 MB 165,2:10am
5,LOLA.2023.1080p.AMZN.WEB-DL.DDP5.1.H.264-WINX[...,7.6 GB 195,2:00am
6,A Royal Christmas Crush 2023 1080p WEB HEVC x265,943.5 MB 118,7pm Jul. 9th
7,Unknown.Killer.Robots.2023.720p.WEBRip.800MB.x...,797.8 MB 26,8:35am
8,Wall-E.2008.Blu-Ray.1080p.x265.12bit.[Hindi-En...,6.7 GB 84,5:36am
9,Heaven.in.Hell.2023.1080p.WEB-DL.H.264.Dual.YG⭐,3.9 GB 66,5pm Jul. 9th


In [ ]:
#@title Run this cell if you want to search the movie
Torrent_name = "casting" #@param {type:"string"}
Torrent_type = "Movies" #@param ["Movies", "Games","Anime", "XXX"]
page = 2 #@param {type:"integer"}
del movies_name[::]
del movies_size_seed[::]
del movies_upload_date[::]
del link[::]
try:
  value=Torrent_name.replace(" ","+")
  url="https://1337x.to/sort-category-search/"+value+"/"+Torrent_type+"/time/desc/"+str(page)+"/"
  r1=requests.get(url)
  soup=BeautifulSoup(r1.content,'html.parser')
  tab1=soup.find('div',class_='table-list-wrap').find('tbody').find_all('tr')
  for i in range(len(tab1)):
    movies_name.append(tab1[i].find_all('td')[0].get_text())
    movies_size_seed.append(tab1[i].find_all('td')[4].get_text().replace("B","B "))
    movies_upload_date.append(tab1[i].find_all('td')[3].get_text())
    temp= soup.find_all('td',class_='coll-1 name')
    link.append(temp[i].find_all('a', href=True)[1]['href'])
  print("RESULT FOUND")

except : print("NO RESULT FOUND")

RESULT FOUND


In [ ]:
#@title Torrent list
data = {'Movie Name': movies_name,
      'Size And Seed': movies_size_seed,
      'Upload':movies_upload_date}
df = pd.DataFrame(data)
data_table.DataTable(df, num_rows_per_page=len(tab1))

In [ ]:
#@title Movie index selector
movie_index = "1 5" #@param {type:"string"}
sub_path = "Other" #@param ["K Drama", "Movie", "Hindi web series", "MARVEL", "Games", "Other"]
select= list(map(int, movie_index.split()))
params = {"save_path": "/content/drive/My Drive/PRAJWAL save/"+sub_path}
for i in select:
  mag_link=[]
  url="https://1337x.to"+link[i]
  r2=requests.get(url)
  soup=BeautifulSoup(r2.content,'html.parser')
  mag_link=(soup.find_all('a',href=True)[30]['href'])
  downloads.append(lt.add_magnet_uri(ses, mag_link, params) )

In [ ]:
#@title Movie Downloader
import time
from IPython.display import display
import ipywidgets as widgets

state_str = [
    "queued",
    "checking",
    "downloading metadata",
    "downloading",
    "finished",
    "seeding",
    "allocating",
    "checking fastresume",
]

layout = widgets.Layout(width="auto")
style = {"description_width": "initial"}
download_bars = [
    widgets.FloatSlider(
        step=0.01, disabled=True, layout=layout, style=style
    )
    for _ in downloads
]
display(*download_bars)

while downloads:
    next_shift = 0
    for index, download in enumerate(downloads[:]):
        bar = download_bars[index + next_shift]
        if not download.is_seed():
            s = download.status()

            bar.description = " ".join(
                [
                    download.name(),
                    str(s.download_rate / 8000),
                    "kB/s",
                    state_str[s.state],
                ]
            )
            bar.value = s.progress * 100
        else:
            next_shift -= 1
            ses.remove_torrent(download)
            downloads.remove(download)
            bar.close() # Seems to be not working in Colab (see https://github.com/googlecolab/colabtools/issues/726#issue-486731758)
            download_bars.remove(bar)
            print(download.name(), "complete")
    time.sleep(1)